In [42]:
import ast
import warnings
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV

In [43]:
warnings.simplefilter(action='ignore')
pd.set_option('display.max_columns', None)

In [2]:
baby_df = pd.read_csv("./babyInfo.csv")
# 데이터 불러오기

In [3]:
baby_growth_df = baby_df.dropna(subset=['growth','gender']) 
# growth, gender 결측치 제거

In [4]:
baby_growth_df['growth'] = baby_growth_df['growth'].apply(ast.literal_eval) 
# 문자열 -> 딕셔너리 형태로 변환
baby_growth_df['height'] = baby_growth_df['growth'].apply(lambda x: x.get('height', {}).get('value', None))
baby_growth_df['weight'] = baby_growth_df['growth'].apply(lambda x: x.get('weight', {}).get('value', None))

C:\Users\82108\AppData\Local\Temp\ipykernel_7232\3092223480.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baby_growth_df['growth'] = baby_growth_df['growth'].apply(ast.literal_eval)
C:\Users\82108\AppData\Local\Temp\ipykernel_7232\3092223480.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baby_growth_df['height'] = baby_growth_df['growth'].apply(lambda x: x.get('height', {}).get('value', None))
C:\Users\82108\AppData\Local\Temp\ipykernel_7232\3092223480.py:4: SettingWithCopyWarning: 
A value is tr

In [5]:
baby_growth_df[baby_growth_df['weight'].isna() & baby_growth_df['height'].isna()] 
# height,weight 둘 다 결측치인 행 확인

,_id,days,desc,gender,__v,growth,height,weight
2239,63490ebc4f30a39628aefdc9,19,어찌저찌 어제보다는 오늘 한결 수월하다!\n그래도 할일은 너무 많고\n너가 울면 어...,M,0,"{'head': {'value': 37, 'unit': 'cm', 'std': 37}}",NaN,NaN
6393,63a96fee0ec395c9ce32feb3,54,NaN,M,0,"{'head': {'value': 39, 'unit': 'cm', 'std': 39}}",NaN,NaN
9350,6325346f204f507df3e5a1b3,78,78일차 아가 성장표\n\n🎀모빌도 좋아하지만 이제는 아기체육관을 더 좋아함\n🎀잘...,F,0,"{'head': {'value': 41, 'unit': 'cm', 'std': 41}}",NaN,NaN
9414,638bd1daffe5f9567ae18982,79,🤎11시간 통잠 4일차 \n🤎밤잠 시작 시간을 7:30pm으로 본인이 정한 듯함\n...,F,0,"{'head': {'value': 39, 'unit': 'cm', 'std': 39}}",NaN,NaN
9920,63639f4e7382e0031c9b5560,83,NaN,M,0,"{'head': {'value': 39.6, 'unit': 'cm', 'std': ...",NaN,NaN
...,...,...,...,...,...,...,...,...
223985,590c7241b9ede03a5c586eb9,131,NaN,M,0,"{'head': {'std': 43, 'value': 43, 'unit': 'cm'}}",NaN,NaN
224346,61080e04babd662a6282d83c,132,NaN,M,0,"{'head': {'value': 42.5, 'unit': 'cm', 'std': ...",NaN,NaN
225178,59bfb0b41787fa4328598fe9,132,NaN,M,0,"{'head': {'unit': 'cm', 'value': 43, 'std': 43}}",NaN,NaN
226391,5a1cb41c01a81b7476aa0877,133,NaN,M,0,"{'head': {'value': 42.9, 'unit': 'cm', 'std': ...",NaN,NaN


In [6]:
baby_growth_df = baby_growth_df.dropna(axis='index', how='all', subset=['height','weight']) 
# 둘 다 결측치인 행 제거

In [7]:
baby_growth_df = baby_growth_df[baby_growth_df['gender'] != 'U'] 
# 성별이 Unknown인 행 삭제

In [8]:
# 'gender' column 추출
gender = baby_growth_df['gender']

# 'gender' column을 one-hot 인코딩하여 새로운 데이터프레임 생성
gender_onehot = pd.get_dummies(gender, prefix='gender')

# 원래 데이터프레임과 합치기
baby_growth_df = pd.concat([baby_growth_df, gender_onehot], axis=1)

# 'gender' column 삭제
baby_growth_df = baby_growth_df.drop('gender', axis=1)

In [9]:
baby_growth_df.sort_values(by='days', inplace=True) # 생후 개월 수로 오름차순 정렬

In [10]:
baby_growth_df['days'] = pd.to_numeric(baby_growth_df['days'])

In [11]:
male_df = baby_growth_df[baby_growth_df['gender_M']==1].copy()
female_df = baby_growth_df[baby_growth_df['gender_F']==1].copy()

In [12]:
male_df.drop(['gender_F'], axis=1, inplace=True)

In [13]:
female_df.drop(['gender_M'], axis=1, inplace=True)

## 결측치 처리 - KNNImputer

In [14]:
#imputer = KNNImputer(n_neighbors=5)

In [15]:
# male_df에서 height 칼럼의 결측치 처리
"""
male_df_height = male_df['height'].values.reshape(-1, 1)
imputed_male_df_height = imputer.fit_transform(male_df_height)
male_df['height'] = imputed_male_df_height

# female_df에서 height 칼럼의 결측치 처리
female_df_height = female_df['height'].values.reshape(-1, 1)
imputed_female_df_height = imputer.fit_transform(female_df_height)
female_df['height'] = imputed_female_df_height
"""

"\nmale_df_height = male_df['height'].values.reshape(-1, 1)\nimputed_male_df_height = imputer.fit_transform(male_df_height)\nmale_df['height'] = imputed_male_df_height\n\n# female_df에서 height 칼럼의 결측치 처리\nfemale_df_height = female_df['height'].values.reshape(-1, 1)\nimputed_female_df_height = imputer.fit_transform(female_df_height)\nfemale_df['height'] = imputed_female_df_height\n"

In [16]:
"""# male_df에서 weight 칼럼의 결측치 처리
male_df_weight = male_df['weight'].values.reshape(-1, 1)
imputed_male_df_weight = imputer.fit_transform(male_df_weight)
male_df['weight'] = imputed_male_df_weight

# female_df에서 weight 칼럼의 결측치 처리
female_df_weight = female_df['weight'].values.reshape(-1, 1)
imputed_female_df_weight = imputer.fit_transform(female_df_height)
female_df['weight'] = imputed_female_df_weight"""

"# male_df에서 weight 칼럼의 결측치 처리\nmale_df_weight = male_df['weight'].values.reshape(-1, 1)\nimputed_male_df_weight = imputer.fit_transform(male_df_weight)\nmale_df['weight'] = imputed_male_df_weight\n\n# female_df에서 weight 칼럼의 결측치 처리\nfemale_df_weight = female_df['weight'].values.reshape(-1, 1)\nimputed_female_df_weight = imputer.fit_transform(female_df_height)\nfemale_df['weight'] = imputed_female_df_weight"

In [17]:
male_df['height'] = male_df.groupby('days')['height'].transform(lambda x: x.fillna(x.median()))
male_df['weight'] = male_df.groupby('days')['weight'].transform(lambda x: x.fillna(x.median()))
female_df['height'] = female_df.groupby('days')['height'].transform(lambda x: x.fillna(x.median()))
female_df['weight'] = female_df.groupby('days')['weight'].transform(lambda x: x.fillna(x.median()))

In [18]:
# 이상치 제거 함수
def remove_outliers(df, col):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5*iqr
    upper_bound = q3 + 1.5*iqr
    df = df.loc[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

In [19]:
# 남성 생후 일 수 별로 그룹화
grouped_male = male_df.groupby('days')

# 각 생후 일 수별로 이상치 제거된 데이터프레임을 저장할 빈 데이터프레임 생성
cleaned_df_male = pd.DataFrame(columns=male_df.columns)

# 생후 일 수(days)별로 이상치 제거된 데이터를 cleaned_df_male에 추가
for name, group in grouped_male:
    group = remove_outliers(group, 'height')
    group = remove_outliers(group, 'weight')
    cleaned_df_male = pd.concat([cleaned_df_male, group])


In [20]:
# 여성 생후 일 수 별로 그룹화
grouped_female = female_df.groupby('days')

# 각 생후 일 수별로 이상치 제거된 데이터프레임을 저장할 빈 데이터프레임 생성
cleaned_df_female = pd.DataFrame(columns=female_df.columns)

# 생후 일 수(days)별로 이상치 제거된 데이터를 cleaned_df_female에 추가
for name, group in grouped_female:
    group = remove_outliers(group, 'height')
    group = remove_outliers(group, 'weight')
    cleaned_df_female = pd.concat([cleaned_df_female, group])


In [21]:
baby_growth_df = pd.concat([cleaned_df_male, cleaned_df_female], ignore_index=True)

In [22]:
baby_growth_df['gender_F'].fillna(0, inplace=True)
baby_growth_df['gender_M'].fillna(0, inplace=True)

In [23]:
baby_growth_df['gender'] = 0
baby_growth_df.loc[baby_growth_df['gender_M'] == True, 'gender'] = 1

In [24]:
baby_growth_df.drop(['gender_M', 'gender_F'], axis = 1, inplace = True)

In [25]:
baby_growth_df

,_id,days,desc,__v,growth,height,weight,gender
0,5ab0230abbd2ac53661aaa2a,0,NaN,0,"{'weight': {'unit': 'kg', 'value': 3.78, 'std'...",52.0,3.78,1
1,5ab023294ba8ac623954eec1,0,청각검사 정상,0,"{'weight': {'unit': 'kg', 'std': 3.78, 'value'...",52.0,3.78,1
2,58e81abdaa513c73561e5b40,0,NaN,0,"{'weight': {'std': 3.2, 'unit': 'kg', 'value':...",50.0,3.20,1
3,5bcba32fb9ede04f165f3ffd,1,NaN,0,"{'weight': {'unit': 'kg', 'std': 2.45, 'value'...",51.0,2.45,1
4,5bc8208036373c1537505616,1,기쁨이가 태어났어요!!\n\n이틀 전부터 진통이 왔는데 5분주기의 짧은 진통은\n안...,0,"{'height': {'std': 50, 'unit': 'cm', 'value': ...",50.0,2.80,1
...,...,...,...,...,...,...,...,...
79344,6272a1c3e0ff637706e8a345,365,NaN,0,"{'weight': {'value': 9.6, 'unit': 'kg', 'std':...",74.6,9.60,0
79345,6225ee63f67e6cf242486ac2,365,"어제 돌잔치를 무사히 끝내고 내일이면 우리 연서의 첫 생일 이구나.. 뱃속 10달,...",0,"{'weight': {'value': 8.6, 'unit': 'kg', 'std':...",74.6,8.60,0
79346,61768ecf383ebb9ee23d044d,365,NaN,0,"{'weight': {'value': 9.4, 'unit': 'kg', 'std':...",74.6,9.40,0
79347,60e7cd722f0e2f96d8e1b099,365,젖병떼기 연습.\n빨대컵에 분유주기 시작.,0,"{'weight': {'value': 9.4, 'unit': 'kg', 'std':...",74.6,9.40,0


## 체중 예측 모델

In [26]:
input_gender, input_days, input_height, input_weight = map(float, input().split(','))

0.0, 40.0, 60.0, 6.0


In [27]:
y_target = baby_growth_df['weight']
x_data = baby_growth_df.drop(['_id', 'desc', '__v', 'growth', 'weight', 'height'], axis = 1)

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_target, test_size = 0.2, stratify = x_data['gender'], random_state=42)

In [29]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(63479, 2) (15870, 2) (63479,) (15870,)


In [30]:
x_data = x_data.astype({'days' : 'int'})

## RandomForestRegressor

In [31]:
params ={
    'n_estimators':[50, 100, 300, 500],
    'max_depth':[8, 12, 16],
    'min_samples_leaf':[4, 8, 12],
    'min_samples_split':[4, 8, 12]
}

rf_model = RandomForestRegressor()
grid_cv = GridSearchCV(rf_model, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(x_train,y_train)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:  {'max_depth': 8, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 50}
최고 예측 정확도: 0.8890


In [32]:
# 모델 학습
rf_model = RandomForestRegressor(n_estimators = 50, max_depth = 8, min_samples_leaf = 4, min_samples_split = 8, random_state=42, oob_score=True)
rf_model.fit(x_train, y_train)

pred = rf_model.predict(x_test)

# 평가
print("훈련 세트 정확도: {:.3f}".format(rf_model.score(x_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(rf_model.score(x_test, y_test)))
print("OOB 샘플의 정확도: {:.3f}".format(rf_model.oob_score_))

훈련 세트 정확도: 0.891
테스트 세트 정확도: 0.888
OOB 샘플의 정확도: 0.890


## LinearRegressor

In [33]:
lr_model = LinearRegression()

lr_model.fit(x_train, y_train)
lr_model.score(x_test, y_test)

print("훈련 세트 정확도: {:.3f}".format(lr_model.score(x_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(lr_model.score(x_test, y_test)))

훈련 세트 정확도: 0.792
테스트 세트 정확도: 0.791


## GradientBoostingRegressor

In [34]:
gbr_model = GradientBoostingRegressor()

gbr_model.fit(x_train, y_train)

# 평가
print("훈련 세트 정확도: {:.3f}".format(gbr_model.score(x_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(gbr_model.score(x_test, y_test)))

훈련 세트 정확도: 0.891
테스트 세트 정확도: 0.888


In [35]:
gbr_model.predict(np.array([input_days+30, input_gender]).reshape(1, -1))

C:\Users\82108\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


array([5.69340705])

In [54]:
graph = []
for i in range(1, 181) :
    graph.append(round(gbr_model.predict(np.array([input_days + i, input_gender]).reshape(1, -1))[0], 2))

In [55]:
graph

[4.76,
 4.81,
 4.83,
 4.9,
 4.97,
 4.98,
 5.0,
 5.0,
 5.03,
 5.1,
 5.12,
 5.16,
 5.19,
 5.23,
 5.28,
 5.28,
 5.31,
 5.31,
 5.39,
 5.39,
 5.47,
 5.48,
 5.56,
 5.57,
 5.57,
 5.6,
 5.6,
 5.67,
 5.67,
 5.69,
 5.69,
 5.76,
 5.77,
 5.77,
 5.83,
 5.89,
 5.89,
 5.96,
 6.0,
 6.0,
 5.99,
 5.99,
 6.08,
 6.11,
 6.15,
 6.18,
 6.22,
 6.22,
 6.22,
 6.29,
 6.29,
 6.33,
 6.33,
 6.33,
 6.35,
 6.35,
 6.46,
 6.46,
 6.5,
 6.5,
 6.5,
 6.58,
 6.58,
 6.65,
 6.65,
 6.65,
 6.65,
 6.65,
 6.66,
 6.76,
 6.76,
 6.76,
 6.76,
 6.76,
 6.76,
 6.82,
 6.89,
 6.89,
 6.97,
 6.97,
 6.97,
 7.0,
 7.04,
 7.04,
 7.04,
 7.04,
 7.04,
 7.09,
 7.14,
 7.14,
 7.15,
 7.15,
 7.19,
 7.19,
 7.19,
 7.19,
 7.2,
 7.2,
 7.2,
 7.2,
 7.2,
 7.2,
 7.25,
 7.25,
 7.41,
 7.41,
 7.42,
 7.42,
 7.42,
 7.42,
 7.48,
 7.48,
 7.48,
 7.48,
 7.51,
 7.51,
 7.47,
 7.49,
 7.49,
 7.49,
 7.49,
 7.49,
 7.6,
 7.6,
 7.6,
 7.6,
 7.74,
 7.85,
 7.85,
 7.85,
 7.85,
 7.85,
 7.85,
 7.85,
 7.85,
 7.85,
 7.85,
 7.85,
 7.85,
 7.85,
 7.95,
 7.95,
 7.95,
 8.02,
 8.02,
 8.02,
